In [ ]:
 # Install necessary libraries
!pip install neo4j openai langchain

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

In [ ]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough

try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [ ]:

os.environ["OPENAI_API_KEY"] = "********"
os.environ["NEO4J_URI"] = "neo4j+s://ef5adff4.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "********"

graph = Neo4jGraph()

In [ ]:
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125") # gpt-4-0125-preview occasionally has issues
llm_transformer = LLMGraphTransformer(llm=llm)
driver = GraphDatabase.driver(uri, auth=(username, password))
def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        return result.data()

# Example query to retrieve all nodes and relationships
query = """
MATCH (n)-[r]->(m)
RETURN n, r, m
"""
data = run_query(query)

# Convert data to JSON format
data_json = json.dumps(data, indent=2)

# Initialize LLM with API key
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125", openai_api_key="sk-svcacct-l53jq7nNGp5kEBA2JlKbU15M8IItd0BbPHsOJsn7F2uuW2dT3BlbkFJGOBUM2kazvV9XZ6-LrwKXmc-DMw42eYEWQ3JShpMyQk2W2wA") # gpt-4-0125-preview occasionally has issues
llm_transformer = LLMGraphTransformer(llm=llm)

In [ ]:


default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri=os.environ["NEO4J_URI"],
        auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph=session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

In [ ]:
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]"
)

# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
                    "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are extracting organization and person entities from the text."),
        ("human", "Use the given format to extract information from the following input: {question}"),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

In [ ]:
entity_chain

ChatPromptTemplate(input_variables=['question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are extracting organization and person entities from the text.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Use the given format to extract information from the following input: {question}'))])
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f48e2564d90>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f48e25666b0>, root_client=<openai.OpenAI object at 0x7f48e2517430>, root_async_client=<openai.AsyncOpenAI object at 0x7f48e2564dc0>, model_name='gpt-3.5-turbo-0125', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy=''), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Entities', 'description': 'Identifying information about entities.', 'parameters': {'type': 'object', 'properties': {'n

In [ ]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = input.split()
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

# Condense a chat history and follow-up question into a standalone question
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""  # noqa: E501
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

_search_query = RunnableBranch(
    (RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(run_name="HasChatHistoryCheck"),
     RunnablePassthrough.assign(chat_history=lambda x: _format_chat_history(x["chat_history"]))
     | CONDENSE_QUESTION_PROMPT
     | ChatOpenAI(temperature=0)
     | StrOutputParser()),
    RunnableLambda(lambda x: x["question"])
)

template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (RunnableParallel({"context": _search_query | retriever, "question": RunnablePassthrough()})
         | prompt
         | llm
         | StrOutputParser())

In [ ]:
chain.invoke({"question": "If a person has chest pain which reports he should get before he visits doctor "})

Search query: If a person has chest pain which reports he should get before he visits doctor 


'The person should get an ECG and blood tests before visiting the doctor.'

In [ ]:
chain.invoke({"question": "if a person has chronic kidney disease which doctor he should visit"})

Search query: if a person has chronic kidney disease which doctor he should visit


'A person with chronic kidney disease should visit a nephrologist.'

In [ ]:
chain.invoke({"question": "If a person has kidney disease recommend a diet to him"})

Search query: If a person has kidney disease recommend a diet to him


'A person with kidney disease should follow a low-protein, low-sodium diet and limit potassium and phosphorus intake. They should also drink plenty of water and avoid processed foods.'